In [66]:
import pandas as pd

temp_list = []

with open("train_call.txt", 'r') as temp:
    for line in temp:
        temp_list.append(line.split())

columns_temp = temp_list[0]
columns_temp = [x.replace("\"", "") for x in columns_temp]

df_train = pd.DataFrame(temp_list[1:], columns=columns_temp)
X = df_train.drop(['Chromosome', 'Start','End', 'Nclone'], axis=1)
X = X.transpose()

labels = []

# X for features Y for breast cancer subtype where 1 = HER2+, 2 = HR+, 3 = Triple Neg
with open("Train_clinical.txt", 'r') as temp_labels:
    next(temp_labels)
    for line in temp_labels:
        temp = line.strip()
        temp = temp.split()
        if temp[1].strip("\"") == "HER2+":
            labels.append(1)
        if temp[1].strip("\"") == "HR+":
            labels.append(2)
        if temp[1].strip("\"") == "Triple":
            labels.append(3)

Y = labels
            
important_genes = []

with open("important_genes.csv", 'r') as imp_genes:
    for line in imp_genes:
        temp = line.strip()
        important_genes.append(temp.split(","))
        
imp_genes = []

for gene in important_genes[1:]:
    imp_genes.append([gene[0], gene[1], gene[2], gene[3]])
    
important_genes = []

for x in imp_genes:
    for i, y in enumerate(df_train.values.tolist()):
        if x[1] == y[0] and x[2] == y[1] and x[3] == y[2]:
            important_genes.append(i)
            
print(X)
print(Y)
print("The regions found in the literature were: ", important_genes)

          0    1    2    3    4    5    6    7    8    9     ... 2824 2825  \
Array.129    0    0    0    0    0    0    0    0    0    0  ...    2    2   
Array.34     0    0    0    0    0    0    0    0    0    0  ...    1    1   
Array.67     0    0    0    0    0    0    0    0    0    0  ...    1    1   
Array.24     0    0    0    0    0    0    0   -1    0    0  ...    0    0   
Array.22     0    0    0    0    0    0    0    0    0    0  ...    1    1   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
Array.10     0    0    0    0    0    0    0    0    0    0  ...    1    1   
Array.123    0    0    0    0    0    0    0    0    0    0  ...    1    1   
Array.100    0    0    0    0    0    0    0    0    0    0  ...    1    1   
Array.134   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1  ...    1    1   
Array.130    0    0    0    0    0    0    0    0    0    0  ...    1    1   

          2826 2827 2828 2829 2830 2831 2832 2833  
Array.129  

In [68]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

# #Loading data
# df = pd.read_csv('Processed_data_2.csv')

# # X for features Y for breast cancer subtype where 1 = HER2+, 2 = HR+, 3 = Triple Neg
# X = df.drop(['Unnamed: 0', 'Sample','Subgroup'], axis=1)
# Y = df['Subgroup']
# print(X)
patients = X.values.tolist()

## FEATURE SELECTION

# we need to shift the features by 1 since the chi2 function does not take non-negative values
patients_shift = []

for patient in patients:
    x = []
    for feature in patient:
        x.append(int(feature) + 1)
    patients_shift.append(x)

chi, p_val = chi2(patients_shift, Y)

imp_genes_p_values = []

for i, j in enumerate(important_genes):
    if p_val[j] >= 0.05:
        imp_genes_p_values.append([imp_genes[i][0], j, p_val[j], "No"])
    else:
        imp_genes_p_values.append([imp_genes[i][0], j, p_val[j], "Yes"])
    

important_genes_scores = pd.DataFrame(imp_genes_p_values, columns=["Gene", "Region number", "P-value", "Yes/No"])
print(important_genes_scores)

X_feature_selected = X

threshold = 0.05
for i, val in enumerate(p_val):
    # this drops the p_values higher than 0.05 that are not in the significant genes list
    if val >= threshold and i not in important_genes:
        X_feature_selected = X_feature_selected.drop([i], axis=1)

                 Gene  Region number   P-value Yes/No
0                AFF2           2822  0.929904     No
1                AKT1           1866  0.553470     No
2                  AR           2794  0.777129     No
3                  AR           2795  0.918872     No
4                 ATM           1583  0.225059     No
5               BRCA1           2223  0.006155    Yes
6               BRCA1           2224  0.010694    Yes
7               BRCA2           1725  0.693197     No
8               BRIP1           2328  0.554967     No
9               BRIP1           2329  0.523246     No
10              BRIP1           2330  0.819148     No
11  CASP8/CTLA4/BARD1            361  0.802287     No
12               CBFB           2016  0.116079     No
13              CCND3            772  0.156822     No
14               CDH1           2021  0.011081    Yes
15             CDKN1B           1611  0.766551     No
16              CHEK2           2733  0.075718     No
17            CYP19A1       

In [69]:
X = X_feature_selected
number_of_features = len(X.values.tolist()[0])
print("Number of features = {}".format(number_of_features))

Number of features = 181


In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

def classification_report_with_accuracy_score(y_true, y_pred):
    "Scorer for the cross validation function"
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) 

def evaluate_model(model, features, labels):
    "Gets the cross validation score using a 5-fold (cv=5) cross validation"
    scores = cross_val_score(model, features, labels, cv=5, scoring=make_scorer(classification_report_with_accuracy_score))
    return scores

In [70]:
# Random Forest

originalclass = []
predictedclass = []

random_forest = RandomForestClassifier(n_estimators=100)
temp = evaluate_model(random_forest, X, Y)
print(classification_report(originalclass, predictedclass)) 

              precision    recall  f1-score   support

           1       0.86      0.94      0.90        32
           2       0.77      0.75      0.76        36
           3       0.73      0.69      0.71        32

    accuracy                           0.79       100
   macro avg       0.79      0.79      0.79       100
weighted avg       0.79      0.79      0.79       100



In [71]:
# Naive Bayes

originalclass = []
predictedclass = []

gnb = GaussianNB()
temp = evaluate_model(gnb, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       1.00      0.81      0.90        32
           2       0.82      0.86      0.84        36
           3       0.72      0.81      0.76        32

    accuracy                           0.83       100
   macro avg       0.85      0.83      0.83       100
weighted avg       0.84      0.83      0.83       100



In [72]:
# Neural network with 1 hidden layer (the hidden layer is as big as the number of features)

originalclass = []
predictedclass = []

mlp = MLPClassifier(solver='lbfgs', activation="relu", alpha=1e-5,
                    hidden_layer_sizes=(number_of_features,), max_iter=1000)

temp = evaluate_model(mlp, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92        32
           2       0.83      0.83      0.83        36
           3       0.79      0.81      0.80        32

    accuracy                           0.85       100
   macro avg       0.85      0.85      0.85       100
weighted avg       0.85      0.85      0.85       100



In [73]:
# Neural network with 2 hidden layer (the hidden layer is as big as the number of features)

originalclass = []
predictedclass = []

mlp = MLPClassifier(solver='lbfgs', activation="relu", alpha=1e-5,
                    hidden_layer_sizes=(number_of_features, number_of_features), max_iter=1000)

temp = evaluate_model(mlp, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       0.91      0.94      0.92        32
           2       0.84      0.86      0.85        36
           3       0.77      0.72      0.74        32

    accuracy                           0.84       100
   macro avg       0.84      0.84      0.84       100
weighted avg       0.84      0.84      0.84       100



In [74]:
# Neural network with 3 hidden layer (the hidden layer is as big as the number of features)

originalclass = []
predictedclass = []

mlp = MLPClassifier(solver='lbfgs', activation="relu", alpha=1e-5,
                    hidden_layer_sizes=(number_of_features, number_of_features, number_of_features,), max_iter=1000)

temp = evaluate_model(mlp, X, Y)
print(classification_report(originalclass, predictedclass))

              precision    recall  f1-score   support

           1       0.83      0.94      0.88        32
           2       0.82      0.86      0.84        36
           3       0.77      0.62      0.69        32

    accuracy                           0.81       100
   macro avg       0.81      0.81      0.80       100
weighted avg       0.81      0.81      0.80       100

